In [1]:
# pip install numpy matplotlib tensorflow tensorflow pickle-mixin scikit-learn

In [2]:
import numpy as np

class NeuralNetwork:
    def __init__(self, N, layer_sizes, lr, activation, weight_init, epochs, batch_size):
        self.N = N  # Number of layers
        self.layer_sizes = layer_sizes  # List of neurons in each layer
        self.lr = lr  # Learning rate
        self.activation = activation  # Activation function
        self.weight_init = weight_init  # Weight initialization method
        self.epochs = epochs  # Number of epochs
        self.batch_size = batch_size  # Batch size

        # Initialize weights and biases
        self.weights, self.biases = self.initialize_weights()

    def initialize_weights(self):
        weights = []
        biases = []

        for i in range(self.N - 1):
            if self.weight_init == "xavier":
                weight = np.random.randn(self.layer_sizes[i], self.layer_sizes[i + 1]) * np.sqrt(1 / self.layer_sizes[i])
            elif self.weight_init == "he":
                weight = np.random.randn(self.layer_sizes[i], self.layer_sizes[i + 1]) * np.sqrt(2 / self.layer_sizes[i])
            else:  # default random initialization
                weight = np.random.randn(self.layer_sizes[i], self.layer_sizes[i + 1]) * 0.01
            bias = np.zeros((1, self.layer_sizes[i + 1]))

            weights.append(weight)
            biases.append(bias)

        return weights, biases

    def activation_function(self, z):
        if self.activation == "relu":
            return np.maximum(0, z)
        elif self.activation == "tanh":
            return np.tanh(z)
        elif self.activation == "sigmoid":
            return 1 / (1 + np.exp(-z))
        else:
            raise ValueError("Unsupported activation function")

    def activation_derivative(self, z):
        if self.activation == "relu":
            return np.where(z > 0, 1, 0)
        elif self.activation == "tanh":
            return 1 - np.tanh(z) ** 2
        elif self.activation == "sigmoid":
            sig = self.activation_function(z)
            return sig * (1 - sig)
        else:
            raise ValueError("Unsupported activation function")

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def forward(self, X):
        activations = [X]
        zs = []

        for i in range(self.N - 2):
            z = np.dot(activations[-1], self.weights[i]) + self.biases[i]
            zs.append(z)
            activation = self.activation_function(z)
            activations.append(activation)

        # Output layer with softmax for probabilities
        z = np.dot(activations[-1], self.weights[-1]) + self.biases[-1]
        zs.append(z)
        activation = self.softmax(z)
        activations.append(activation)

        return activations, zs

    def backward(self, X, Y, activations, zs):
        grads_w = [None] * (self.N - 1)
        grads_b = [None] * (self.N - 1)

        # Output layer error
        delta = activations[-1] - Y  # Assuming Y is one-hot encoded
        grads_w[-1] = np.dot(activations[-2].T, delta) / X.shape[0]
        grads_b[-1] = np.sum(delta, axis=0, keepdims=True) / X.shape[0]

        # Backpropagation through hidden layers
        for i in range(self.N - 3, -1, -1):
            delta = np.dot(delta, self.weights[i + 1].T) * self.activation_derivative(zs[i])
            grads_w[i] = np.dot(activations[i].T, delta) / X.shape[0]
            grads_b[i] = np.sum(delta, axis=0, keepdims=True) / X.shape[0]

        return grads_w, grads_b

    def update_parameters(self, grads_w, grads_b):
        for i in range(self.N - 1):
            self.weights[i] -= self.lr * grads_w[i]
            self.biases[i] -= self.lr * grads_b[i]

    def fit(self, X, Y, X_val=None, Y_val=None, early_stopping=False, patience=10):
        epoch_loss = None  # Initialize the loss variable
        best_val_loss = float('inf')  # Initialize best validation loss
        wait = 0  # Initialize wait counter for early stopping

        for epoch in range(self.epochs):
            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)

            for start in range(0, X.shape[0], self.batch_size):
                end = start + self.batch_size
                batch_indices = indices[start:end]
                X_batch, Y_batch = X[batch_indices], Y[batch_indices]

                activations, zs = self.forward(X_batch)
                grads_w, grads_b = self.backward(X_batch, Y_batch, activations, zs)
                self.update_parameters(grads_w, grads_b)

            # Calculate training loss for the current epoch
            full_activations, _ = self.forward(X)
            epoch_loss = -np.mean(np.sum(Y * np.log(full_activations[-1] + 1e-8), axis=1))

            # Validation loss calculation
            if X_val is not None and Y_val is not None:
                val_predictions = self.predict_proba(X_val)
                val_loss = -np.mean(np.sum(Y_val * np.log(val_predictions + 1e-8), axis=1))
                print(f"Epoch {epoch + 1}/{self.epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

                # Early stopping check
                if early_stopping:
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        wait = 0  # Reset the wait counter
                    else:
                        wait += 1
                        if wait >= patience:
                            print(f"Early stopping triggered at epoch {epoch + 1}")
                            break
            else:
                print(f"Epoch {epoch + 1}/{self.epochs}, Train Loss: {epoch_loss:.4f}")

        return epoch_loss  # Return the calculated epoch loss

    def predict(self, X):
        activations, _ = self.forward(X)
        return np.argmax(activations[-1], axis=1)

    def predict_proba(self, X):
        activations, _ = self.forward(X)
        return activations[-1]

    def score(self, X, Y):
        predictions = self.predict(X)
        labels = np.argmax(Y, axis=1)  # Assuming Y is one-hot encoded
        accuracy = np.mean(predictions == labels)
        return accuracy

In [3]:
import numpy as np

class ActivationFunctions:
    
    @staticmethod
    def sigmoid(z):
        """
        Sigmoid activation function.
        """
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def sigmoid_derivative(z):
        """
        Derivative of the sigmoid function.
        """
        sig = ActivationFunctions.sigmoid(z)
        return sig * (1 - sig)

    @staticmethod
    def tanh(z):
        """
        Tanh activation function.
        """
        return np.tanh(z)

    @staticmethod
    def tanh_derivative(z):
        """
        Derivative of the tanh function.
        """
        return 1 - np.tanh(z) ** 2

    @staticmethod
    def relu(z):
        """
        ReLU activation function.
        """
        return np.maximum(0, z)

    @staticmethod
    def relu_derivative(z):
        """
        Derivative of the ReLU function.
        """
        return np.where(z > 0, 1, 0)

    @staticmethod
    def leaky_relu(z, alpha=0.01):
        """
        Leaky ReLU activation function with a small slope for negative inputs.
        """
        return np.where(z > 0, z, alpha * z)

    @staticmethod
    def leaky_relu_derivative(z, alpha=0.01):
        """
        Derivative of the Leaky ReLU function.
        """
        return np.where(z > 0, 1, alpha)

    @staticmethod
    def softmax(z):
        """
        Softmax activation function for the output layer.
        """
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability improvement
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [4]:
import numpy as np

class WeightInitializer:
    
    @staticmethod
    def zero_init(shape):
        """
        Initializes weights to zero.
        """
        return np.zeros(shape)
    
    @staticmethod
    def random_init(shape, scale=0.01):
        """
        Initializes weights randomly within a uniform distribution.
        """
        return np.random.uniform(-scale, scale, shape)
    
    @staticmethod
    def normal_init(shape, scale=1.0):
        """
        Initializes weights using a normal distribution with mean 0 and standard deviation scale.
        """
        return np.random.normal(0, scale, shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# Configuration dictionary
configurations = {
    "num_layers": 5,
    "layer_sizes": [784, 256, 128, 64, 10],
    "learning_rate": 0.1,
    "epochs": 100,
    "batch_size": 128,
    "activation_functions": ["sigmoid", "tanh", "relu", "leaky_relu"],
    "weight_initializations": ["zero_init", "random_init", "normal_init"]
}

# Load and preprocess the MNIST dataset
(X, y), (X_test, y_test) = mnist.load_data()
X = X.reshape(-1, 784) / 255.0
X_test = X_test.reshape(-1, 784) / 255.0
Y = np.eye(10)[y]
Y_test = np.eye(10)[y_test]

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

# Store training and validation losses for plotting
training_losses = {}
validation_losses = {}

for activation in configurations["activation_functions"]:
    for weight_init in configurations["weight_initializations"]:
        
        print(f"Training with activation: {activation} and weight initialization: {weight_init}")
        
        model = NeuralNetwork(
            N=configurations["num_layers"],
            layer_sizes=configurations["layer_sizes"],
            lr=configurations["learning_rate"],
            activation=activation,
            weight_init=weight_init,
            epochs=configurations["epochs"],
            batch_size=configurations["batch_size"]
        )

        train_loss_history = []
        val_loss_history = []
        
        for epoch in range(configurations["epochs"]):
            # Training the model with early stopping enabled
            train_loss = model.fit(X_train, Y_train, X_val=X_val, Y_val=Y_val, early_stopping=True, patience=10)
            val_predictions = model.predict_proba(X_val)
            val_loss = -np.mean(np.sum(Y_val * np.log(val_predictions + 1e-8), axis=1)) if val_predictions is not None else None
            
            # Append losses to history lists
            train_loss_history.append(train_loss)
            val_loss_history.append(val_loss)
            
            # Print epoch progress with conditional handling of None values
            print(f"Epoch {epoch + 1}/{configurations['epochs']} - Train Loss: {train_loss if train_loss is not None else 'N/A'} - Val Loss: {val_loss if val_loss is not None else 'N/A'}")
        
        training_losses[(activation, weight_init)] = train_loss_history
        validation_losses[(activation, weight_init)] = val_loss_history
        
        model_filename = f"model_{activation}_{weight_init}.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(model, f)
        
# Plot training and validation loss for each configuration
for (activation, weight_init), train_loss_history in training_losses.items():
    val_loss_history = validation_losses[(activation, weight_init)]
    plt.figure(figsize=(10, 5))
    plt.plot(train_loss_history, label="Train Loss")
    plt.plot(val_loss_history, label="Validation Loss")
    plt.title(f"Activation: {activation}, Weight Init: {weight_init}")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

2024-11-10 08:50:39.910974: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 08:50:39.914753: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 08:50:39.922989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731228639.935360 1460595 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731228639.939129 1460595 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 08:50:39.954576: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Training with activation: sigmoid and weight initialization: zero_init
Epoch 1/100, Train Loss: 2.3025, Val Loss: 2.3021
Epoch 2/100, Train Loss: 2.3040, Val Loss: 2.3049
Epoch 3/100, Train Loss: 2.3054, Val Loss: 2.3063
Epoch 4/100, Train Loss: 2.3035, Val Loss: 2.3050
Epoch 5/100, Train Loss: 2.3024, Val Loss: 2.3038
Epoch 6/100, Train Loss: 2.3020, Val Loss: 2.3033
Epoch 7/100, Train Loss: 2.3027, Val Loss: 2.3042
Epoch 8/100, Train Loss: 2.3023, Val Loss: 2.3032
Epoch 9/100, Train Loss: 2.3037, Val Loss: 2.3042
Epoch 10/100, Train Loss: 2.3032, Val Loss: 2.3028
Epoch 11/100, Train Loss: 2.3018, Val Loss: 2.3023
Early stopping triggered at epoch 11
Epoch 1/100 - Train Loss: 2.301806188464979 - Val Loss: 2.302292349397229
Epoch 1/100, Train Loss: 2.3040, Val Loss: 2.3028
Epoch 2/100, Train Loss: 2.3022, Val Loss: 2.3031
Epoch 3/100, Train Loss: 2.3019, Val Loss: 2.3033
Epoch 4/100, Train Loss: 2.3027, Val Loss: 2.3028
Epoch 5/100, Train Loss: 2.3017, Val Loss: 2.3025
Epoch 6/100, Tra